In [42]:
import numpy as np
import os, re, pandas as pd
import tqdm

from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint
from scipy.optimize import LinearConstraint
from scipy.optimize import Bounds

from matplotlib import pyplot as plt

### Functions

In [45]:
def entropy_production(energies, p, beta, return_q=False):
    S_in = np.array([1-p, p]).reshape(2, 1)
    dB = energies.size

    B_in = np.exp(-beta * energies) / np.sum(np.exp(-beta * energies))
    SB_in = np.ravel(np.kron(S_in, B_in))

    SB_out = np.sort(SB_in)[::-1]
    SB_out_reshaped = SB_out.reshape((dB, 2), order='F')

    S_out = np.sum(SB_out_reshaped, axis=0)
    B_out = np.sum(SB_out_reshaped, axis=1)

    dEb = np.sum((B_out - B_in) * energies)
    dS = np.sum(sc.special.entr(S_out)) - np.sum(sc.special.entr(S_in))
    Wirr = dEb + dS

    if return_q:
        return (Wirr, S_out[1])
    else:
        return Wirr

def load_all_Wirr(output_dir="opt_results"):
    Wirr_values = []
    Ns = []

    # Match files like: opt_result_N3.csv, opt_result_N10.csv, etc.
    pattern = re.compile(r"opt_result_N(\d+)\.csv")

    for filename in os.listdir(output_dir):
        match = pattern.match(filename)
        if match:
            N = int(match.group(1))
            filepath = os.path.join(output_dir, filename)
            df = pd.read_csv(filepath)
            Wirr = df['Wirr'].iloc[0]
            Ns.append(N)
            Wirr_values.append((N, Wirr))

    Wirr_values = np.array(Wirr_values)
    Wirr_sorted = Wirr_values[Wirr_values[:, 0].argsort()]
    return Wirr_sorted

def load_all_energies(output_dir="opt_results"):
    energy_data = []
    pattern = re.compile(r"opt_result_N(\d+)\.csv")

    for filename in os.listdir(output_dir):
        match = pattern.match(filename)
        if match:
            N = int(match.group(1))
            filepath = os.path.join(output_dir, filename)
            df = pd.read_csv(filepath)

            # Extract columns like 'E0', 'E1', ...
            energy_cols = [col for col in df.columns if col.startswith('E')]

            # Convert string tuple to real tuple (energy, degeneracy)
            energies = [eval(df[col].iloc[0]) for col in energy_cols]
            energy_data.append((N, energies))

    energy_data.sort(key=lambda x: x[0])
    return energy_data  # List of (N, [(energy, degeneracy), ...])


    param_data = []
    pattern = re.compile(r"opt_result_N(\d+)\.csv")

    for filename in os.listdir(output_dir):
        match = pattern.match(filename)
        if match:
            N = int(match.group(1))
            filepath = os.path.join(output_dir, filename)
            df = pd.read_csv(filepath)

            # Extract parameter columns starting with 'x'
            param_cols = [col for col in df.columns if col.startswith('x')]
            params = [df[col].iloc[0] for col in param_cols]
            param_data.append((N, params))

    # Sort by N
    param_data.sort(key=lambda x: x[0])
    return param_data  # List of tuples: (N, [x0, x1, ...])
    
def full_energy_spectrum(spectrum):
    full_spectrum = []
    for energy, degeneracy in spectrum:
        full_spectrum.extend([energy] * degeneracy)
    return np.array(full_spectrum)
    
def optimize_model(model_energies, N, q, output_dir="opt_results"):
    os.makedirs(output_dir, exist_ok=True)

    def model_entprod(params, model_energies, N):
        energies = model_energies(N, params)
        return entropy_production(full_energy_spectrum(energies), p=0.5, beta=1.0)

    def model_constraint(params, model_energies, N, q):
        energies = model_energies(N, params)
        _, q_actual = entropy_production(full_energy_spectrum(energies), p=0.5, beta=1.0, return_q=True)
        return q - q_actual

    i = 0
    Wirr_opt = 0
    
    x0 = [1.0] * N
    x0 = np.random.rand(N)
        
    bounds = [(0, None)] * N

    goal_fun = lambda params: model_entprod(params, model_energies, N)
    constraint = {
            'type': 'ineq',
            'fun': lambda params: model_constraint(params, model_energies, N, q)
    }

    result = minimize(
        goal_fun,
        x0=x0,
        bounds=bounds,
        constraints=[constraint],
        method='COBYLA',
        options={'maxiter': 30000, 'rhobeg': 1.0, 'tol': 1e-12}
    )
    
    return result

### Parameters of the protocol

In [46]:
p = 0.5
q = 1e-5
beta = 1

### Model 1: Uniform star-graph network

In [47]:
def uniform_star_energies(N, params):
    """Compute the energy levels of the Star model for N spins."""
    a = params[0]
    b = params[1]
    
    energies = []

    # First group: σ1 = +1, energy depends on configuration of other N-1 spins
    for k in range(N):  # k = number of up spins among N-1
        energy = a + 2 * b * (2 * k - (N - 1))  # Energy for σ1 = +1
        degeneracy = int(comb(N - 1, k))
        energies.append((energy, degeneracy))

    # Second group: σ1 = -1, energy is just -a, highly degenerate
    energy = -a
    degeneracy = int(2 ** (N - 1))
    energies.append((energy, degeneracy))

    return energies

In [48]:
N = 5
n_sims = 10

params = np.zeros((n_sims,2))
energies = {}
Wirr = np.zeros((n_sims,))

for i in tqdm.tqdm(range(n_sims)):
    result = optimize_model(uniform_star_energies, N, q)

    Wirr[i] = result.fun
    params[i] = result.x[0:2]
    energies[i] = uniform_star_energies(N, params[i])

# Choose best
best_sim = np.argmin(Wirr)
Wirr_best = Wirr[best_sim]
params_best = params[best_sim]
energies_best = energies[best_sim]

print(f'Wirr_best = {Wirr_best}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.80it/s]

Wirr_best = 2.787069181183451


In [49]:
# Save corresponding datapoint to file
output_dir = 'uniform_weights'
os.makedirs(output_dir, exist_ok=True)

result_data = [Wirr_best] + list(params_best) + list(energies_best)
header = ['Wirr'] + [f'x{i}' for i in range(1, 3)] + [f'E{i}' for i in range(0, N+1)]
df = pd.DataFrame([result_data], columns=header)
df.to_csv(os.path.join(output_dir, f"opt_result_N{N}.csv"), index=False)

In [62]:
# load file for plotting
Wirr_uniform_star = load_all_Wirr(output_dir="uniform_weights")

### Model 2: Nonuniform star-graph network

In [53]:
def nonuniform_star_energies(N, params):
    """
    Energy spectrum of a star network with non-uniform couplings b_i to spin 1.
    """
    a = params[0]
    b_list = params[1:]
        
    assert len(b_list) == N - 1
    energies = {}

    for i in range(2**(N-1)):
        config = [(i >> k) & 1 for k in range(N-1)]  # binary config of N-1 spins
        spin_vals = [2*s - 1 for s in config]  # convert 0/1 -> -1/+1
        interaction = sum(b * s for b, s in zip(b_list, spin_vals))

        # σ₁ = +1
        energy_plus = a + interaction
        energies[energy_plus] = energies.get(energy_plus, 0) + 1

        # σ₁ = -1
        energy_minus = -a + interaction
        energies[energy_minus] = energies.get(energy_minus, 0) + 1

    return sorted(energies.items())

In [55]:
N = 5
n_sims = 5

params = {}
energies = {}
Wirr = np.zeros((n_sims,))

for i in tqdm.tqdm(range(n_sims)):
    result = optimize_model(nonuniform_star_energies, N, q)

    Wirr[i] = result.fun
    params[i] = result.x
    energies[i] = nonuniform_star_energies(N, params[i])

# Choose best
best_sim = np.argmin(Wirr)
Wirr_best = Wirr[best_sim]
params_best = params[best_sim]
energies_best = energies[best_sim]

print(f'Wirr_best = {Wirr_best}')

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.63it/s]

Wirr_best = 0.30137267751113905


In [56]:
# save corresponding datapoint to file
output_dir = 'non_uniform_weights'
os.makedirs(output_dir, exist_ok=True)

result_data = [Wirr_best] + list(params_best) + list(energies_best)
header = ['Wirr'] + [f'x{i}' for i in range(len(params_best))] + [f'E{i}' for i in range(len(energies_best))]
df = pd.DataFrame([result_data], columns=header)
df.to_csv(os.path.join(output_dir, f"opt_result_N{N}.csv"), index=False)

In [61]:
# load file for plotting
Wirr_non_uniform_star = load_all_Wirr(output_dir="non_uniform_weights")

### Plotting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import LogFormatterSciNotation, ScalarFormatter

# Constants
Nmin = 2
Nmax = 15
Npoints = Nmax - Nmin

# Placeholder data (replace with your actual data)
# Wirr_uniform_star = np.array([...])
# Wirr_non_uniform_star = np.array([...])

x = Wirr_non_uniform_star[:Npoints, 0]
y1 = Wirr_uniform_star[:Npoints, 1]
y2 = Wirr_non_uniform_star[:Npoints, 1]

# Analytic bounds
y3 = (1.0 / 3.0) * (1.0 / x)
y4 = (1.0 / np.log(2)) * (1.0 / (x**2))

logy1 = np.log10(y1)
logy2 = np.log10(y2)
logy3 = np.log10(y3)
logy4 = np.log10(y4)

# Extrapolation parameters
Nmax_inter = 200
n_inter = 5

x_inter = np.arange(x[-1], Nmax_inter)
y3_ext = (1.0 / 3.0) * (1.0 / x_inter)
y4_ext = (1.0 / np.log(2)) * (1.0 / (x_inter**2))
logy3_ext = np.log10(y3_ext)
logy4_ext = np.log10(y4_ext)

# Linear extrapolation in log-log space
fit1_ext = np.polyfit(np.log10(x[-n_inter:]), logy1[-n_inter:], deg=1)
fit2_ext = np.polyfit(np.log10(x[-n_inter:]), logy2[-n_inter:], deg=1)

logy1_ext = np.polyval(fit1_ext, np.log10(x_inter))
logy2_ext = np.polyval(fit2_ext, np.log10(x_inter))

# Colors
color1 = mcolors.TABLEAU_COLORS['tab:blue']
color2 = mcolors.TABLEAU_COLORS['tab:red']
color3 = mcolors.TABLEAU_COLORS['tab:green']
color4 = mcolors.TABLEAU_COLORS['tab:purple']

# Plotting
fig = plt.figure()
ax = plt.gca()

ax.set_xscale('log')
ax.set_yscale('log')

# Plot main curves
ax.plot(x, y1, label='STAR (uniform)', c=color3, lw=3)
ax.plot(x, y2, label='STAR (non-uniform)', c=color4, lw=3)

# Plot extrapolated curves
ax.plot(x_inter, 10**logy1_ext, c=color3, lw=3, linestyle='dotted')
ax.plot(x_inter, 10**logy2_ext, c=color4, lw=3, linestyle='dotted')

# Plot analytic bounds
ax.plot(x, y3, c=color1, lw=3, linestyle='dashed')
ax.plot(x, y4, c=color2, lw=3, linestyle='dashed')
ax.plot(x_inter, y3_ext, c=color1, lw=3, linestyle='dashed')
ax.plot(x_inter, y4_ext, c=color2, lw=3, linestyle='dashed')

# Plot fit lines in original region
initial_point = 5
fit1 = np.polyfit(np.log10(x[initial_point:]), logy1[initial_point:], deg=1)
fit2 = np.polyfit(np.log10(x[initial_point:]), logy2[initial_point:], deg=1)

fit_line1 = np.polyval(fit1, np.log10(x[initial_point:]))
fit_line2 = np.polyval(fit2, np.log10(x[initial_point:]))

# ax.plot(x[initial_point:], 10**fit_line1, color='black', linestyle='dashed')
# ax.plot(x[initial_point:], 10**fit_line2, color='black', linestyle='dashed')

# Fill analytic region
all_x = np.concatenate([x, x_inter])
all_y3 = np.concatenate([y3, y3_ext])
all_y4 = np.concatenate([y4, y4_ext])

ax.fill_between(all_x, all_y4, all_y3, where=all_y3 >= all_y4, facecolor=color2, alpha=0.05, interpolate=True)
ax.fill_between(all_x, all_y3, 100, where=1 >= all_y3, facecolor=color1, alpha=0.05, interpolate=True)

# Labels and legend
ax.set_xlabel('Number of qubits (n)')
ax.set_ylabel(r'Entropy production ($\Sigma$)')
ax.legend(loc='lower left', fontsize=32, frameon=False)

# Format x-axis to scientific notation
ax.xaxis.set_major_formatter(LogFormatterSciNotation(base=10))
ax.yaxis.set_major_formatter(LogFormatterSciNotation(base=10))

# Set specific ticks for y-axis (e.g., 10^-1, 10^-3, 10^-5)
specific_ticks = [10**1, 10**-1, 10**-3, 10**-5]
ax.set_yticks(specific_ticks)

ax.set_xlim([2,199]) 
ax.set_ylim([10E-6,10])

plt.tight_layout()
plt.savefig('plot.pdf', dpi=600)
plt.show()
